In [4]:
!pip install allennlp
!pip install azure.cognitiveservices.language.textanalytics

     |████████████████████████████████| 43 kB 1.6 MB/s  eta 0:00:01


In [6]:
import nltk
import json
import pandas as pd
from allennlp.predictors.predictor import Predictor
import re
import string
from collections import Counter
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
import numpy as np
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/anonymous/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
#Download PreTrained QA System Model
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz")

100%|██████████| 46175392/46175392 [00:00<00:00, 46505798.74B/s]
/anaconda/envs/py37_default/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/anaconda/envs/py37_default/lib/python3.7/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:56: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/anaconda/envs/py37_default/lib/python3.7/site-packages/en_core_web_sm -->
/anaconda/envs/py37_default/lib/python3.7/site-packages/spacy/data/en_core_web_sm
You can now load the model via spacy.load('en_core_web_sm')


In [11]:
#Read Data file
data=pd.read_csv('file.csv')

In [12]:
data.head()

,url,title,body,summary
0,https://www.linkedin.com/feed/news/states-to-s...,States to set their reopening plans,Reopening the U.S. economy will take place in ...,Reopening the U.S. economy will take place in ...
1,https://www.linkedin.com/feed/news/china-ends-...,China ends its growth streak,The world's second largest economy officially ...,The world's second largest economy officially ...
2,https://www.linkedin.com/feed/news/big-tech-hi...,Big Tech hints at workforce changes,Some of the world's biggest tech companies are...,Some of the world's biggest tech companies are...
3,https://www.linkedin.com/feed/news/workshare-p...,Workshare programs grow in demand,"Texas, Oregon, Colorado, Wisconsin and Arkansa...","Texas, Oregon, Colorado, Wisconsin and Arkansa..."
4,https://www.linkedin.com/feed/news/can-the-cru...,Can the cruise industry survive?,The world’s biggest cruise ship companies are ...,The world’s biggest cruise ship companies are ...


In [13]:
data.describe()

,url,title,body,summary
count,29232,29227,26353,26353
unique,20242,19083,14390,14025
top,https://food.ndtv.com/lists/10-best-basmati-ri...,"Business News, Economic News, Market News, Sha...","CNN name, logo and all associated elements ® a...","CNN name, logo and all associated elements ® a..."
freq,15,58,1975,1975


In [14]:
def clean_text(data):
    clean1 = re.sub(r'['+string.punctuation + '’—”'+']', "", data.lower())
    return re.sub(r'\W+', ' ', clean1)

In [20]:
data['paragraphs'] = data['body'].map(lambda x: clean_text(str(x)))
data['title'] = data['title'].map(lambda x: clean_text(str(x)))

In [21]:
data.head()

,url,title,body,summary,paragraphs
0,https://www.linkedin.com/feed/news/states-to-s...,states to set their reopening plans,Reopening the U.S. economy will take place in ...,Reopening the U.S. economy will take place in ...,reopening the us economy will take place in th...
1,https://www.linkedin.com/feed/news/china-ends-...,china ends its growth streak,The world's second largest economy officially ...,The world's second largest economy officially ...,the worlds second largest economy officially s...
2,https://www.linkedin.com/feed/news/big-tech-hi...,big tech hints at workforce changes,Some of the world's biggest tech companies are...,Some of the world's biggest tech companies are...,some of the worlds biggest tech companies are ...
3,https://www.linkedin.com/feed/news/workshare-p...,workshare programs grow in demand,"Texas, Oregon, Colorado, Wisconsin and Arkansa...","Texas, Oregon, Colorado, Wisconsin and Arkansa...",texas oregon colorado wisconsin and arkansas a...
4,https://www.linkedin.com/feed/news/can-the-cru...,can the cruise industry survive,The world’s biggest cruise ship companies are ...,The world’s biggest cruise ship companies are ...,the worlds biggest cruise ship companies are f...


In [23]:
data.dropna()
data.paragraphs.dropna()
data.title.dropna()

0                      states to set their reopening plans
1                             china ends its growth streak
2                      big tech hints at workforce changes
3                        workshare programs grow in demand
4                          can the cruise industry survive
                               ...                        
29227    karun chandok view from the paddock motorsport...
29228                        get your digital subscription
29229                                         john cherian
29230                                   purnima s tripathi
29231                                      t k rajalakshmi
Name: title, Length: 29232, dtype: object

In [24]:
#find our desired words in an articles
def find_cc_wds(content, cc_wds=['jobs','remote','job loss','layoff','layoffs','layoffs in industry','work from home','unemployed','unemployment','reskilling']
):
    found = False
    for w in cc_wds:
        if w in content:
            found = True
            break

    if not found:
        disj = re.compile(r'(package\w+\W+(?:\w+\W+){1,5}?government) | (scheme\W+(?:\w+\W+){1,5}?government)')
        if disj.match(content):
            found = True
    return found

In [25]:
#find our desired words in title of the article
data['wds'] = data['title'].apply(find_cc_wds)
data['wds'].head()

0    False
1    False
2    False
3    False
4    False
Name: wds, dtype: bool

In [26]:
data2 = data[data['wds']==True] 
data2.reset_index(drop=True, inplace=True)
data2

,url,title,body,summary,paragraphs,wds
0,https://www.linkedin.com/feed/news/remote-work...,remote work hiring jumps,The coronavirus pandemic has sent the work-fro...,The coronavirus pandemic has sent the work-fro...,the coronavirus pandemic has sent the workfrom...,True
1,https://news.linkedin.com/2020/march/most-in-d...,most indemand jobs during coronavirus,"Last week, LinkedIn’s Chief Economist shared h...","LinkedIn data shows this outlook is correct, t...",last week linkedins chief economist shared her...,True
2,https://hindi.thequint.com/news/education,education news in hindiश क ष सम च र jobs and c...,लॉकडाउन के बाद कभी भी आ सकता है पश्चिम बंगाल ब...,लॉकडाउन के बाद कभी भी आ सकता है पश्चिम बंगाल ब...,ल कड उन क ब द कभ भ आ सकत ह पश च म ब ग ल ब र ड ...,True
3,https://www.thequint.com/me-the-change/me-the-...,me the change kshitija wants more jobs for cre...,Kshiija Sarda is a 22-year-old woman from Pune...,Kshiija Sarda is a 22-year-old woman from Pune...,kshiija sarda is a 22yearold woman from pune m...,True
4,https://www.ndtv.com/india-news/coronavirus-lo...,no interstate travel for migrant workers must ...,The lockdown badly affected the daily-wage mig...,The lockdown badly affected the daily-wage mig...,the lockdown badly affected the dailywage migr...,True
...,...,...,...,...,...,...
169,https://www.ndtv.com/india-news/coronavirus-gu...,gurgaon multinationals may have to work from h...,"Gurgaon, which is part of the NCR, is known as...",The Gurgaon district administration had issued...,gurgaon which is part of the ncr is known as t...,True
170,https://khabar.ndtv.com/news/jobs?pfrom=home-k...,job news in hindi government jobs employment n...,"Listen to the latest songs , only on JioSaavn.com","Listen to the latest songs , only on JioSaavn.com",listen to the latest songs only on jiosaavncom,True
171,https://khabar.ndtv.com/news/jobs?pfrom=home-jobs,job news in hindi government jobs employment n...,"Listen to the latest songs , only on JioSaavn.com","Listen to the latest songs , only on JioSaavn.com",listen to the latest songs only on jiosaavncom,True
172,https://indianexpress.com/article/trending/tre...,stunt professionals work from home to choreogr...,The viral video featured students and instruct...,Watch the video hereThe cleverly edited video ...,the viral video featured students and instruct...,True


In [27]:
#clean summary of data
data2['paragraph'] = data2['summary'].map(lambda x: clean_text(str(x)))
data2

/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,url,title,body,summary,paragraphs,wds,paragraph
0,https://www.linkedin.com/feed/news/remote-work...,remote work hiring jumps,The coronavirus pandemic has sent the work-fro...,The coronavirus pandemic has sent the work-fro...,the coronavirus pandemic has sent the workfrom...,True,the coronavirus pandemic has sent the workfrom...
1,https://news.linkedin.com/2020/march/most-in-d...,most indemand jobs during coronavirus,"Last week, LinkedIn’s Chief Economist shared h...","LinkedIn data shows this outlook is correct, t...",last week linkedins chief economist shared her...,True,linkedin data shows this outlook is correct th...
2,https://hindi.thequint.com/news/education,education news in hindiश क ष सम च र jobs and c...,लॉकडाउन के बाद कभी भी आ सकता है पश्चिम बंगाल ब...,लॉकडाउन के बाद कभी भी आ सकता है पश्चिम बंगाल ब...,ल कड उन क ब द कभ भ आ सकत ह पश च म ब ग ल ब र ड ...,True,ल कड उन क ब द कभ भ आ सकत ह पश च म ब ग ल ब र ड ...
3,https://www.thequint.com/me-the-change/me-the-...,me the change kshitija wants more jobs for cre...,Kshiija Sarda is a 22-year-old woman from Pune...,Kshiija Sarda is a 22-year-old woman from Pune...,kshiija sarda is a 22yearold woman from pune m...,True,kshiija sarda is a 22yearold woman from pune m...
4,https://www.ndtv.com/india-news/coronavirus-lo...,no interstate travel for migrant workers must ...,The lockdown badly affected the daily-wage mig...,The lockdown badly affected the daily-wage mig...,the lockdown badly affected the dailywage migr...,True,the lockdown badly affected the dailywage migr...
...,...,...,...,...,...,...,...
169,https://www.ndtv.com/india-news/coronavirus-gu...,gurgaon multinationals may have to work from h...,"Gurgaon, which is part of the NCR, is known as...",The Gurgaon district administration had issued...,gurgaon which is part of the ncr is known as t...,True,the gurgaon district administration had issued...
170,https://khabar.ndtv.com/news/jobs?pfrom=home-k...,job news in hindi government jobs employment n...,"Listen to the latest songs , only on JioSaavn.com","Listen to the latest songs , only on JioSaavn.com",listen to the latest songs only on jiosaavncom,True,listen to the latest songs only on jiosaavncom
171,https://khabar.ndtv.com/news/jobs?pfrom=home-jobs,job news in hindi government jobs employment n...,"Listen to the latest songs , only on JioSaavn.com","Listen to the latest songs , only on JioSaavn.com",listen to the latest songs only on jiosaavncom,True,listen to the latest songs only on jiosaavncom
172,https://indianexpress.com/article/trending/tre...,stunt professionals work from home to choreogr...,The viral video featured students and instruct...,Watch the video hereThe cleverly edited video ...,the viral video featured students and instruct...,True,watch the video herethe cleverly edited video ...


In [29]:
#SETTING KEY AND ENDPOINT OF AZURE COGNITIVE SERVICE- TEXT ANALYTICS
key = '79737d5112134bb88bd34d6840750ebc'
endpoint = 'https://newsscrapper.cognitiveservices.azure.com/'

In [30]:
#AUTHENTICATE THE KEY AND ENDPOINT FROM AZURE

def authenticateClient():
    credentials = CognitiveServicesCredentials(key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint, credentials=credentials)
    return text_analytics_client

In [31]:
# ENTITY RECOGNITION IN DOCUMENTS

def entity_recognition(documents):
    
    client = authenticateClient()

    try:

        response = client.entities(documents=documents)

        for document in response.documents:
            print("Document Id: ", document.id)
            print("\tKey Entities:")
            for entity in document.entities:
                print("\t\t", "NAME: ", entity.name, "\tType: ",
                      entity.type, "\tSub-type: ", entity.sub_type)
                for match in entity.matches:
                    print("\t\t\tOffset: ", match.offset, "\tLength: ", match.length, "\tScore: ",
                          "{:.2f}".format(match.entity_type_score))

    except Exception as err:
        print("Encountered exception. {}".format(err))

In [32]:
#KEY PHRASES IN THE DOCUMENTS

def key_phrases(documents):
    
    client = authenticateClient()

    try:

        for document in documents:
            print(
                "Asking key-phrases on '{}' (id: {})".format(document['text'], document['id']))

        response = client.key_phrases(documents=documents)

        for document in response.documents:
            print("Document Id: ", document.id)
            print("\tKey Phrases:")
            for phrase in document.key_phrases:
                print("\t\t", phrase)

    except Exception as err:
        print("Encountered exception. {}".format(err))

In [33]:
#SENTIMENT ANALYSIS ON THE DOCUMENT

def sentiment(documents):
    
    client = authenticateClient()

    try:
        response = client.sentiment(documents=documents)
        for document in response.documents:
            print("Document Id: ", document.id, ", Sentiment Score: ",
                  "{:.2f}".format(document.score))

    except Exception as err:
        print("Encountered exception. {}".format(err))

In [28]:
#formatting the summary to fit in azure functions
x=[]
for i in data2.index:
    info={"id": i+1,"text":data2.paragraph[i]}
    x.append(info)
print(x)

[{'id': 1, 'text': 'the coronavirus pandemic has sent the workfromhome trend into light speed with millions around the world suddenly forced to work from their home office or more likely their dining table once considered a perk remote work jobs have exploded now out of necessity and managers have often gone from simply tolerating the arrangement to embracing it says linkedin senior editoratlarge george anders linkedin found that job postings for remote work jumped 28 in march and job searches involving the keywords remote or work at home were up 42 the job titles enjoying the greatest increase in marchs remotework postings include software engineer account executive institutional sales and more'}, {'id': 2, 'text': 'linkedin data shows this outlook is correct the us is heading into a period of uncertainty as employers quickly change hiring strategies and pivot the way they sell goods and services linkedin data shows a demand for workers in roles that are on the frontlines of helping u

In [34]:
#Calling functions on data
entity_recognition(x) #LIVE ENTITY RECOGNITION ON NEWS ARTICLES OF A NEWSPAPER
sentiment(x) #LIVE SENTIMENT ANALYSIS ON NEWS ARTICLES OF A NEWSPAPER
key_phrases(x)#LIVE KEY PHRASES EXTRACTION OF ARTICLES OF A NEWSPAPER

Document Id:  1
	Key Entities:
		 NAME:  now 	Type:  DateTime 	Sub-type:  None
			Offset:  240 	Length:  3 	Score:  0.80
		 NAME:  LinkedIn 	Type:  Organization 	Sub-type:  None
			Offset:  350 	Length:  8 	Score:  0.80
			Offset:  394 	Length:  8 	Score:  0.80
		 NAME:  28 in 	Type:  Quantity 	Sub-type:  Dimension
			Offset:  450 	Length:  5 	Score:  0.80
		 NAME:  march 	Type:  DateTime 	Sub-type:  DateRange
			Offset:  456 	Length:  5 	Score:  0.80
		 NAME:  42 	Type:  Quantity 	Sub-type:  Number
			Offset:  533 	Length:  2 	Score:  0.80
Document Id:  2
	Key Entities:
		 NAME:  LinkedIn 	Type:  Organization 	Sub-type:  None
			Offset:  0 	Length:  8 	Score:  0.80
			Offset:  184 	Length:  8 	Score:  0.80
			Offset:  409 	Length:  8 	Score:  0.80
			Offset:  672 	Length:  8 	Score:  0.80
		 NAME:  the us 	Type:  Location 	Sub-type:  None
			Offset:  44 	Length:  6 	Score:  0.65
			Offset:  431 	Length:  6 	Score:  0.65
			Offset:  499 	Length:  6 	Score:  0.65
			Offset:  620 	Length

In [37]:
#find our desired words in articles
data['cc_wds'] = data['paragraphs'].apply(find_cc_wds)
data['cc_wds'].head()

0    False
1    False
2    False
3     True
4    False
Name: cc_wds, dtype: bool

In [38]:
data.head()

,url,title,body,summary,paragraphs,wds,cc_wds
0,https://www.linkedin.com/feed/news/states-to-s...,states to set their reopening plans,Reopening the U.S. economy will take place in ...,Reopening the U.S. economy will take place in ...,reopening the us economy will take place in th...,False,False
1,https://www.linkedin.com/feed/news/china-ends-...,china ends its growth streak,The world's second largest economy officially ...,The world's second largest economy officially ...,the worlds second largest economy officially s...,False,False
2,https://www.linkedin.com/feed/news/big-tech-hi...,big tech hints at workforce changes,Some of the world's biggest tech companies are...,Some of the world's biggest tech companies are...,some of the worlds biggest tech companies are ...,False,False
3,https://www.linkedin.com/feed/news/workshare-p...,workshare programs grow in demand,"Texas, Oregon, Colorado, Wisconsin and Arkansa...","Texas, Oregon, Colorado, Wisconsin and Arkansa...",texas oregon colorado wisconsin and arkansas a...,False,True
4,https://www.linkedin.com/feed/news/can-the-cru...,can the cruise industry survive,The world’s biggest cruise ship companies are ...,The world’s biggest cruise ship companies are ...,the worlds biggest cruise ship companies are f...,False,False


In [39]:
data1 = data[data['cc_wds']==True] 
data1.reset_index(drop=True, inplace=True)
data1

,url,title,body,summary,paragraphs,wds,cc_wds
0,https://www.linkedin.com/feed/news/workshare-p...,workshare programs grow in demand,"Texas, Oregon, Colorado, Wisconsin and Arkansa...","Texas, Oregon, Colorado, Wisconsin and Arkansa...",texas oregon colorado wisconsin and arkansas a...,False,True
1,https://www.linkedin.com/feed/news/remote-work...,remote work hiring jumps,The coronavirus pandemic has sent the work-fro...,The coronavirus pandemic has sent the work-fro...,the coronavirus pandemic has sent the workfrom...,True,True
2,https://www.linkedin.com/feed/news/heres-whos-...,heres whos hiring right now,Some industries — from shipping to online lear...,Some industries — from shipping to online lear...,some industries from shipping to online learni...,False,True
3,https://www.linkedin.com/pulse/how-covid-19-im...,how covid19 is impacting hiring around the world,Countries around the world are in different st...,About two weeks after we saw hiring growth con...,countries around the world are in different st...,False,True
4,https://news.linkedin.com/2020/march/most-in-d...,most indemand jobs during coronavirus,"Last week, LinkedIn’s Chief Economist shared h...","LinkedIn data shows this outlook is correct, t...",last week linkedins chief economist shared her...,True,True
...,...,...,...,...,...,...,...
1643,https://indianexpress.com/article/india/tablig...,not right to fault a community over actions of...,Mohan Bhagwat. (File Photo) Mohan Bhagwat. (Fi...,Mohan Bhagwat.\n(File Photo) Mohan Bhagwat.\nB...,mohan bhagwat file photo mohan bhagwat file ph...,False,True
1644,https://indianexpress.com/article/business/eco...,report by irs officers seeks tax hikes for tac...,Finance Ministry officials claimed the report ...,Finance Ministry officials claimed the report ...,finance ministry officials claimed the report ...,False,True
1645,https://indianexpress.com/article/parenting/le...,five apps that will keep kids busy in this per...,You can keep your kids busy with certain apps ...,You can keep your kids busy with certain apps ...,you can keep your kids busy with certain apps ...,False,True
1646,https://malayalam.indianexpress.com/jobs/sbi-c...,ല ക ക ഡ ൺ എസ ബ ഐ ക ലർക ക മ യ ൻ പര ക ഷ മ റ റ യ ...,കൊറോണ വൈറസിനെ തുടർന്ന് രാജ്യത്ത് ലോക്ക്ഡൗൺ പ്ര...,കൊറോണ വൈറസിനെ തുടർന്ന് രാജ്യത്ത് ലോക്ക്ഡൗൺ പ്ര...,ക റ ണ വ റസ ന ത ടർന ന ര ജ യത ത ല ക ക ഡ ൺ പ രഖ യ...,False,True


In [43]:
len(data1)
len(data2)

174

In [47]:
data1['paragraphs']

0       texas oregon colorado wisconsin and arkansas a...
1       the coronavirus pandemic has sent the workfrom...
2       some industries from shipping to online learni...
3       countries around the world are in different st...
4       last week linkedins chief economist shared her...
                              ...                        
1643    mohan bhagwat file photo mohan bhagwat file ph...
1644    finance ministry officials claimed the report ...
1645    you can keep your kids busy with certain apps ...
1646    ക റ ണ വ റസ ന ത ടർന ന ര ജ യത ത ല ക ക ഡ ൺ പ രഖ യ...
1647    jadavpur mp mimi chakraborty will interact wit...
Name: paragraphs, Length: 1648, dtype: object

In [53]:
out = ' '.join(data1["paragraphs"].head(20))
print (out)

texas oregon colorado wisconsin and arkansas are among the states seeing a rise in businesses applying for workshare programs since the start of the pandemic the staterun programs which let companies reduce hours without having to layoff employees means workers can collect prorated unemployment benefits some 52 million americans filed initial unemployment claims between april 5 and april 11 according to the department of labor the claims add to the nearly 17 million new unemployment applications that have been filed since the coronavirus pandemic forced businesses to shutter across the us  the coronavirus pandemic has sent the workfromhome trend into light speed with millions around the world suddenly forced to work from their home office or more likely their dining table once considered a perk remote work jobs have exploded now out of necessity and managers have often gone from simply tolerating the arrangement to embracing it says linkedin senior editoratlarge george anders linkedin 

In [54]:
result=predictor.predict(
  passage=out,
  question="which industries will face layoffs and job loss?"
)
result['best_span_str']

'mining manufacturing and real estate and so there will not be any drastic change in infrastructure and creation of jobs instead of taking proactive measures to revive economic growth the budget seems to to be content with giving a push to the stakeholders of the economy investors and entrepreneurs to spur economic growth the government has removed around 70 exemptions in the new income tax regime in order to simplify the process and ensure that citizens are able to file their tax returns without the help of any advisers the personal tax changes the minister claims will result in a loss of rs 40000 crore to the exchequer the finance minister has scrapped the dividend distribution tax which could see a loss of revenue the government hasnt thought of structural factors instead it appears to be in denial about the economy being demandconstrained and investmentstarved the idea of reforms seems restricted to giving small dollops of tax relief to tax payers where were you when the mandal com